In [2]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
from http.client import responses

import requests
from mpmath.math2 import math_sqrt

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")
MODEL    = os.getenv("MODEL_NAME", "bens_model")

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 128,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


# Three of thought: Second inference method


In [3]:
class Node:
    def __init__(self, thought, children = None):
        self.thought = thought
        self.children = children or []

class Tot:
    def __init__(self, prompt, model = None, iterations = 3, maxTokens = 128):
        self.root = Node(prompt)
        self.model = model or MODEL
        self.iterations = iterations
        self.current_thoughts = [self.root]
        self.maxTokens = maxTokens


    def EThoughts(self, thoughtN):
        newThoughtNodes = []

        for nd in thoughtN:
            prompt = f"Given the current thought: '{nd.thought}', provide three concise thoughts that expand this idea further."
            response = call_model_chat_completions(prompt)


            if response:
                child = Node(response["text"])
                nd.children.append(child)
                newThoughtNodes.append(child)

        return newThoughtNodes


    def Run(self):
        iteration = 0
        while self.current_thoughts and iteration < self.iterations:
            print(f"Iteration {iteration + 1}:")
            self.current_thoughts = self.EThoughts(self.current_thoughts)

            for tNode in self.current_thoughts:
                print(f"Explored Thought: {tNode.thought}")
            iteration += 1


    def updateThought(self, nThought):
        self.root = Node(nThought)
        self.current_thoughts = [self.root]


    def showTree(self, Node, lv=0, prefix=""):
        connector = "└── " if lv > 0 else ""
        print(f"{prefix}{connector}{Node.thought}")

        # Prepare indentation for children
        child_prefix = prefix + ("    " if lv > 0 else "")

        for child in Node.children:
            self.showTree(child, lv + 1, child_prefix)


if __name__ == "__main__":
    starting_prompt = "what is 2 + 2"
    Tot = Tot(starting_prompt)
    Tot.Run()
    print("=" * 100)
    print("Final Tree of Thoughts:")
    Tot.showTree(Tot.root)


Iteration 1:
Explored Thought: The sum of 2 + 2 equals 4.  
This simple arithmetic can represent the combination of two pairs.  
In programming, 2 + 2 might be used in a calculation or logic statement.
Iteration 2:
Explored Thought: The concept of addition extends beyond numbers, symbolizing union in data structures like arrays or objects.  
In mathematics, 2 + 2 = 4 is foundational to understanding more complex operations and equations.  
In everyday life, the idea of combining two entities reflects principles of collaboration and resource pooling.
Iteration 3:
Explored Thought: The principle of addition mirrors the merging of datasets in database operations, enhancing information integration.  
In programming, concatenation of strings or merging of lists exemplifies addition's role in data manipulation.  
Socially, the act of combining efforts in a team mirrors addition, leading to collective outcomes greater than individual contributions.
Final Tree of Thoughts:
what is 2 + 2
└── Th

# self-consistency : Third inference method






In [60]:
from collections import Counter

def determineProblem(prompt):
    problemType = call_model_chat_completions(prompt, system= "Determine what type of problem this is and return one of the following choices "
                                                              "[Math reasoning, common sense, logic]")
    return problemType["text"]

def selfConsistency(prompt, num_responses = 5):
    responses = []
    for _ in range(num_responses):

        ## funny way to rem temperature: When ur girlfriend is hot"mad" she is going to say a lot more and complain. When she is cold she will be relaxed and say less amen
        response = call_model_chat_completions(prompt, system = f"Before solving Response{_} must be written down", temperature= .8)
        responses.append(response["text"])

    formatFinalAnswers = responses
    #formatFinalAnswers  = "\n---\n".join([f"{key}:\n{value}" for key, value in responses])

    selectionPrompt = f"""

    I have generated several responses for the given problem. Please evaluate them and determine which is the most accurate, comprehensive, and well written"

    Here are the response:
    {formatFinalAnswers}

    Review all candidate responses, determine which one is the strongest, and reply using *ONLY* label (e.g., “Response 1”) followed by the answer.
    """

    goldenSolution = call_model_chat_completions(selectionPrompt)

    return goldenSolution["text"], formatFinalAnswers

    # finalAnswers = []
    # for res in responses:
    #     match = re.search(r"\*\*Answer: (\d+)", res)
    #     if match:
    #         finalAnswers.append(int(match.group(1)))
    #
    # print(f"All extracted answers: {finalAnswers}")
    #
    # # majority vote
    # if finalAnswers:
    #     VoteResult = Counter(finalAnswers).most_common(1)[0]
    #     print(f"Most consistent answer: {VoteResult[0]} (appeared {VoteResult[1]} times)")
    #     return VoteResult
    #
    # return None

    ##return responses


def runSelfConsistency(prompt):
    problemType = determineProblem(prompt)
    print(problemType)
    Answers = selfConsistency(prompt, num_responses= 5)

    print("The golden solution is", Answers[0], "\n")

    for i in Answers[1]:
        print(i)




prompt = "if there are three cars in the parking lot and 289 more cars arrive, how many cars are in the parking lot"
runSelfConsistency(prompt)

#
# prompt = "if there are three cars in the parking lot and 289 more cars arrive, how many cars are in the parking lot"
# problemType = determineProblem(prompt)
# print(problemType)
#
# Answers = selfConsistency(prompt, num_responses= 5)
# #print(Answers[0])
#
#
# print("The golden solution is", Answers[0])
# for i in Answers[1]:
#     print(i)







Math reasoning
The golden solution is Response 4

**Final Answer: 292 cars** are in the parking lot. 

Response0: 3 + 289 = 292 cars

There are 292 cars in the parking lot.
First, let's solve the problem:

There are initially **3 cars** in the parking lot.  
Then, **289 more cars** arrive.

To find the total number of cars, we add:

$$
3 + 289 = 292
$$

**Answer:** There are **292 cars** in the parking lot.

---

**Response1:** There are 292 cars in the parking lot.
First, let's write down the response to the question:

**Response2:**  
There are 3 cars initially in the parking lot. Then, 289 more cars arrive.  
To find the total number of cars, we add the two amounts together:

$$
3 + 289 = 292
$$

So, there are **292 cars** in the parking lot.
First, let's write down the response:

**Response3: 3 + 289 = 292**

Now, solving the problem:

There are initially 3 cars in the parking lot. Then, 289 more cars arrive.

So, the total number of cars in the parking lot is:

**3 + 289 = 292**



# Agent loop

Action route:
1. Math or logic problems: self-consistency
2. complex problems strategic planning, exploration, and backtracking: tree of thought
3.